In [ ]:
!pip install python-docx
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from google.colab import files
from docx import Document
from docx.shared import Inches

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

x_data = data['Serial Number'].values
y_data = data['Cites'].values

def equation(params, x):
    a, b, c = params
    return (b / (x + c)) - a

def mse_loss(params, x, y):
    y_pred = equation(params, x)
    return np.mean((y - y_pred) ** 2)

initial_params = [1.0, 1.0, 1.0]

result = minimize(mse_loss, initial_params, args=(x_data, y_data), bounds=[(0, None), (0, None), (0, None)])
a_opt, b_opt, c_opt = result.x

fitted_values = equation([a_opt, b_opt, c_opt], x_data)

data['fitted'] = fitted_values
data['a'] = a_opt
data['b'] = b_opt
data['c'] = c_opt

output_file = 'fitted_citation_data.csv'
data.to_csv(output_file, index=False)

plt.scatter(x_data, y_data, label='Original Data')
plt.plot(x_data, fitted_values, color='red', label='Fitted Values')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

plot_file = 'fitted_plot.png'
plt.savefig(plot_file)
plt.show()

doc = Document()
doc.add_heading('Fitted Citation Data', 0)
doc.add_picture(plot_file, width=Inches(6))
doc.add_paragraph(f'Optimized parameters:\na = {a_opt}\nb = {b_opt}\nc = {c_opt}')

doc_file = 'fitted_citation_data.docx'
doc.save(doc_file)

files.download(output_file)
files.download(doc_file)
